In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
# Загрузка данных из файла
data = pd.read_csv("C:/Users/bondarenkovv/Desktop/Python/ML/dataset_fot_toxic_comment/labeled.csv", sep=",")

# Посмотрим на первые несколько строк данных
print(data.head())

                                             comment  toxic
0               Верблюдов-то за что? Дебилы, бл...\n    1.0
1  Хохлы, это отдушина затюканого россиянина, мол...    1.0
2                          Собаке - собачья смерть\n    1.0
3  Страницу обнови, дебил. Это тоже не оскорблени...    1.0
4  тебя не убедил 6-страничный пдф в том, что Скр...    1.0


In [3]:
# Предобработка текста
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bondarenkovv\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bondarenkovv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bondarenkovv\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:

toxic_patterns = ['а поебаться тебе не завернуть?','а сто хуёв — большая куча?','апчхуй','архипиздрит',
                  'ахуй','ахулиард','беден как Адам, и хуя прикрыть нечем','без пизды','бля','блядво',
                  'блядёшка','блядина','блядища','блядки','блядовать','блядовитый','блядоёбина',
                  'блядский','блядство','блядствовать','блядун','блядь','блядькать','блядькнуть','бляха-муха',
                  'босый хуй','босым хуем около голой пизды шутки не шутят','буква «ю» — по хую','был бы хуй, а подхуйки найдутся',
                  'в ахуе','в два хуя','в душе не ебу','в еблет наклацать','в жопу ёбаный','в пизду','в рай въехать на чужом хую',
                  'в рот ебаться','в хуй дует','в хуй дышит','в хуй не всраться','в хуй не дуть','в хуй не ставить','в хуй не упереться',
                  'веселоебучесть','веселоебучий','весь в поте и хуй в роте','весь в поту и хуй во рту','вешать лапшу на хуй',
                  'взъёб','взъебать','взъёбка','взъебти','взъёбывать','Викисловарь:Инкубатор/нахуйник',
                  'влындить хуй','во рту мухи ебутся','война — хуйня, главное — манёвры','вола ебать','волоёб',
                  'волоёбство','вот и весь хуй','вот те нате — хуй в томате!','врубить съебаторы','всё в ажуре, а хуй на абажуре',
                  'всё пучком и пизда торчком','вхуяриться','въёб','въебать','въебаться','въебашить','въебенить','въёбка',
                  'въёбывать','въехать в рай на чужом хую','выблядок','выебанный','выебать','выебаться','выебина','выёбистый',
                  'выебнуть','выебнуться','выебон','выебонщик','выебсти','выебти','выёбывать','выёбываться','выеть','выпиздеться',
                  'выпиздить','гавкнуть пиздой','где совесть была, там хуй вырос','головка от хуя','голодная пизда',
                  'голосуй — не голосуй, всё равно получишь хуй','голохуевка','горит пизда','дать в еблетку',
                  'дать пизды','дать по ебалу','дать по пизде мешалкой','дебилы, блядь','девятый хуй без соли доедать',
                  'дневальный! подай станок ебальный!','до пизды','до хуища','до хуя','довыёбываться','доёб','доебать',
                  'доебаться','доёбистый','доёбка','доебсти','доебти','доёбчивость','доёбчивый','доёбщик','доёбывать',
                  'доёбываться','доигрался хуй на скрипке','долбоёб','долбоебизм','долбоёбина','долбоёбище','долбоёбский',
                  'долбоёбство','допиздеться','дохулион','дохуя','драпиздон','драпиздончик','друг дружку ебут и деньги в кружку кладут',
                  'дуроёб','дядеёб','ёб','ёб вашу мать','ёб иху мать','ёб твою мать','ёб твоя мать','ебабельность','ебабельный',
                  'ебак','ебака','Ебаков','ебал','ебал я тебя в грызло','ебала','ебала жаба гадюку','ебалай','ебалайствовать',
                  'ёбалды','ебало','ебало завали!','ебало на ноль!','ебало офнуть','ебало раскурочить','ебало сверни!',
                  'ебало свернуть','ебальная','ебальник','ебальник начистить','ебальник просрать','ебальник разрисовать',
                  'ебальничек','ебальный','ебальня','ебальце','ебан','ебанарий','ебанат','ебанатик','ебанашка','ебание',
                  'ебанизм','ебанина','ебанистика','ебанистический','ёбанный','ебано','ёбаногандонный хуепедераст',
                  'ебаноид','ебанутость','ебанутый','ебануть','ебануть в бороду','ебануться','ебанушка','ебаный','ёбаный',
                  'ёбаный в жопу','ёбаный в рот','ёбаный насос','ёбаный покос','ёбаный стос','ёбаный стыд','ёбаный шашлык',
                  'ёбань','ебанье','ебанько','ебарёк','ебарёчек','ебаришка','ебаришко','ёбарь','ёбарь-перехватчик','ебати',
                  'ебатись','ебаторий','ебатура','ебать','ебать вола','ебать и резать','ебать мозги','ебать мой лысый череп',
                  'ебать мой хуй ментовской жопой','ебать раком','ебаться','ебаться в телевизор','ебач','ебашить','ебашиться',
                  'ебейший','ебель','ебельник','ебёна мать','ебёна темя','ебенить','ебеный','ебёный','ебеня','еби его мать',
                  'еби её мать','ебись о плетень','ебись оно всё конём','ебись твою в ноздрю','ебит твою мать','ебический',
                  'Ебишка','ёбка','ёбкий','ёбко','ёбкость','еблан','ебланить','ебланище','ебланский','еблет','еблетень',
                  'еблетка','еблец','еблецо','ебливенький','ебливость','ебливый','еблина','еблито','еблишко','еблишко прикрой!',
                  'еблище','ебло','ебло стянуть','ебловать','ёблышко','ебля','ёбля','ёбнутый','ебнуть','ёбнуть','ёбнуть по фазе',
                  'ёбнуть фазу','ёбнуться','ебня','ебобо','ебовый','ебоглазить','ебомый','ебосос','ебосос расхлестать',
                  'ебота','ебота на еботе','ёботность','ёботный','еботня','ебошить','ебошиться','ёбс','ёбс-переёбс','ёбство',
                  'ебсти','ебстись','ебти','ебтись','ёбу даться','ебукать','ебуки','ебун','ебунеть','ебунец','ебунок','ебунья',
                  'ебур','ебут вашу мать','ебут и фамилию не спрашивают','ебучесть','ебучий','ебучка','ебушатник',
                  'ебушатник прикрыть','ебушатня','ёбушки-воробушки','ёбфренд','ёбче','ёбши','ёбывать','ебырок',
                  'ёбырь','ёбыч','ёпт','ёпть','если бы у бабушки был хуй, то она была бы дедушкой','естись','етись','етить',
                  'еть','еться','жевать большой хуище','жевать хуи','жрать нехуя и съебаться некуда','за всю хуйню',
                  'за каким хуем','за не хуй делать','за три пизды','за хуем','забить хуй','заблядовать','заёб','заёба','заёбанный',
                  'заебательский','заебато','заебатый','заебать','заебаться','заебашить','заебенить','заебенить по гражданке',
                  'заебёшься пыль глотать','заёбисто','заёбистый','заебись','заебок','заебти','заебу — больно ходить будет',
                  'заебунеть','заёбушка','заебца','заебцовый','заёбывать','заеть','закрыть пиздак','залупистый','запиздеть',
                  'запиздеться','запиздонить','запиздюливать','запиздюривать','запиздючить','заработать хуем по затылку',
                  'засандалить хуй','захуевертить','захуёвничать','захуяривать','захуярить','захуячить','захуячиться',
                  'зачесалася пизда','защеканец','заябать','збс','здрахуйте','злоебучесть','злоебучий','и в хуй не дуть',
                  'и рыбку съесть, и на хуй сесть','ибиомать','иди на хуй','идти на хуй','изволоёбить','изъёб','изъебать',
                  'изъеблася','изъебнуться','изъебтись','или рыбку съесть, или на хуй сесть','испиздеться','испиздить',
                  'исхуёвничаться','исхуярить','исхуячить','к ебене матери','к ебеней матери','к ебеням','к херам собачьим',
                  'к хуям','к хуям собачьим','каждый кулик на своём хую велик','как из пизды на лыжах','как не хуй делать',
                  'как у латыша — хуй да душа','как хуем сбрило','как хуем сдуло','как хуй с плеч упал','какая пизда тебя родила',
                  'какого хуя','картавый рыжий хуй','кидать через хуй','килька в томате да пизда в халате','кинуть через хуй',
                  'класть крепкий хуй','кого ебёт чужое горе','колдоёбина','кому хуй, а мне всегда два!','коноёбить','коноёбиться',
                  'короёбочка','косоёбить','кремлядь','крыть хуями','кусай за хуй!','къебенизация','ложь, пиздёж и провокация',
                  'мать его ети','мать их ёб','мать иху ёб','мать твою ёб','мать твою распроеби','мёртвого за хуй тянуть',
                  'милипиздрический','мозги ебать','мозгоблуд','мозгоёб','мозгоебатель','мозгоебательство','мозгоёбка',
                  'мозгоёбство','мозгоёбствовать','молодой хуёк','мудоёб','на воре и шапка пизженая','на кой хуй',
                  'на одном месте не уебёшь — надо перетаскивать','на отъебись','на пизду надеяться','на хуище',
                  'на хуй','на хуй — не на гвоздь','на хуй нужен','на хуй нужно','на хуй с пляжа','на хуй твоя жопа хороша',
                  'на хую вертеть','на хуя','на хуях таскать','навернуть еблет','навешать пиздюлей','навставлять пиздюлей',
                  'надавать пиздюлей','наёб','наебалово','наебательство','наебать','наебаться','наебашить','наебениваться',
                  'наебениться','наебизнес','наёбка','наебнуть','наебнуться','наебнуться головой','наеборезиться',
                  'наеборезываться','наебти','наебтись','наёбщик','наёбщица','наёбывать','наихуёвейший','напиздеть',
                  'напиздеться','напиздить','напудрить пизду','нас ебут, а мы крепчаем','настоебать','настопиздеть',
                  'нах','нахохлить пизду','нахуевертить','нахуй','нахуя','нахуярить','нахуяриться','нахуячиваться',
                  'нахуячить','нахуячиться','начистить ебальник','не въебаться','не ебаться','не еби гусей',
                  'не наебёшь ― не проживёшь','не по хуйне','не смеши пизду, она и так смешная','не ставить ни в хуй',
                  'не считать хуй за мясо','не хочешь кулеш, хуй ешь','не хуй','не хуй делать','не хуй делать, пол топтать',
                  'не хуй собачий','не хуя','невзъебенный','невъебенно','невъебенность','невъебенный','недоёб',
                  'неебабельность','неебабельный','неебёный','неебический','нехуёвый','нехуй','нехуйно','нехуйный','нехуя',
                  'ни бороды, ни усов, ни на хую волосов','ни в пизду, ни в Красную Армию','ни в хуй не ставить','ни за хуй',
                  'ни за хуй собачий','ни с хуя','ни хуя','ни хуя себе','ни хуя себе уха','ни хуя уха','нищеёб',
                  'ноль целых, хуй десятых','носим ношеное, ебём брошенное','о пизде ни слова','обложить хуями',
                  'обувать пизду в лапти','объёб','объебать','объебаться','объебашивать','объебашить','объеблан',
                  'объебон','объебос','объебошивать','объебошить','объёбывать','овердохуя','овцеёб','один хуй',
                  'однохуйственно','одолбоёбиться','опездал','опездол','опизденевать','опизденеть','опизденительность',
                  'опизденительный','опиздоволоситься','опиздохуительный','опиздюливать','опиздюливаться','опиздюлить',
                  'опиздюлиться','ослоёб','остаться с хуем','остоебенить','остопиздеть','остопиздить','остохуеть',
                  'от не хуй делать','от хуя кончики','от хуя уши','отпиздеться','отпиздить','отпиздиться','отпиздохать',
                  'отпиздякать','отпиздячить','отхватить пиздюлей','отхуесосить','отхуякать','отхуярить','отхуячивать',
                  'отхуячить','отъебать','отъебаться','отъебнуть','отъебошить','отъебти','отъебукать','отъёбываться',
                  'охуев','охуевание','охуевать','охуевоз','охуевший','охуение','охуенно','охуенность','охуенный',
                  'охуенский','охуенчик','охуетительный','охуеть','охуительно','охуительный','охуй','охулиард','охулион',
                  'охуячивать','охуячить','пёзды','передок','переёб','переебать','переебти','перепиздеть','перехуярить',
                  'перехуячить','персидского царя хуилище','пизда','пизда горит','пизда на ободах','пизда нестроевая',
                  'пизда пилоткой','пизда рулю','пизда с зубами','пизда с ушами','пизда чешется','пиздак','пиздануть',
                  'пиздануться','пиздарики','пиздарики на воздушном шарике','пиздатее','пиздато','пиздатый','пиздёж',
                  'пизделушка','пиздёнка','пиздёночка','пиздёныш','пиздёнышка','пиздень','пиздеть',
                  'пиздеть — не кули ворочать, — спина не болит','пиздеться','пиздец','пиздец подкрался незаметно',
                  'пиздецки','пиздецкий','пиздецовый','пиздёшечка','пиздёшка','пизди, пизди: приятно слушать',
                  'пиздий','пиздиловка','пиздить','пиздиться','пиздишка','пиздища','пиздище','пиздливый','пиздлявый',
                  'пиздлявый ящик','пиздня','пиздоблядство','пиздобол','пиздоболить','пиздобольство','пиздобратия',
                  'пиздовать','пиздоглазый','пиздоделие','пиздодельный','пиздоёбство','пиздой мух ловить',
                  'пиздой накрыться','пиздоквак','пиздолиз','пиздолизка','пиздомудство','пиздопроёбище',
                  'пиздопротивный','пиздорванец','пиздорванец хуев','пиздорванка','пиздорез','пиздоремонтник',
                  'пиздоремонтничек','пиздос','пиздосий','пиздострадалец','пиздохаться','пиздоход','пиздохранилище',
                  'пиздошить','пизду в лапти обувать','пиздуй','пиздун','пиздуна запускать','пиздуна запустить',
                  'пиздушечка','пиздушка','пизды дать','пиздык','пиздюк','пиздюлей вложить','пиздюлей навешать',
                  'пиздюлей навставлять','пиздюлей отхватить','пиздюлина','пиздюль','пиздюря','пиздюхать','пиздюшка',
                  'пиздякнуться','пиздянка','пиздятина','пиздятинка','пиздячить','пиздящий','пизже','пизженый','пинать хуи',
                  'пинать хуй','по ебалу получить','по пизде мешалкой','по пизде мешалкой словить','по хую','повъёбывать',
                  'повыёбываться','под ногти выебать','подарок — из пизды огарок','подзаебать','подзаебаться','подзаебти',
                  'подохуевший','подохуеть','подпизживать','подхуек','подъёб','подъебать','подъебаться','подъебаш','Подъебишкин',
                  'подъёбка','подъебнуть','подъебон','подъебончик','подъебушка','подъёбщик','подъёбывать','Подъебышкин',
                  'поебатор','поебать','поёбать','поебаться','поебень','поёбка','поебок','поебота','поеботина','поебстись',
                  'поебти','поебтись','поебусенка','поебуха','поебушечка','поебушка','поёбывать','пойти по пизде',
                  'показать хуёв тачку','показать хуй','показать хуй огромный','полный похуист','положить хуй',
                  'полпизды','получить крупный хуй','получить от хуя уши','получить пизды','получить хуй','полхуя',
                  'понеслась пизда по кочкам','понтоёб','понтоёбский','поперехуярить','поперехуячить','попизделки',
                  'попиздеть','попиздистее','попиздить','попиздиться','попиздовать','попизже','послать в пизду',
                  'последний хуй без соли доедать','посылать в пизду','потреблядство','пох','похеру','похуизм','похуист',
                  'похуистка','похуй','похуй пидору чулки','похую','похуястее','похуястей','пошароёбиться','пошли на хуй',
                  'приебаться','приёбываться','прикидывать хуй к носу','прикинуть хуй к носу','прикрыть ебушатник','припиздень',
                  'припиздеть','припиздить','припиздок','припиздюхать','прихуеть','прихуяривать','прихуярить','про пизду ни слова',
                  'проблядь','проёб','проебав','проёбанный','проебать','проебаться','проебланить','проебти','проёбчик','проёбщик',
                  'проёбщица','проёбывать','проёбываться','пропиздеть','пропиздеться','пропиздон','проскакивать между ёбаных',
                  'проскочить между ёбаных','протягивать пизду','прыгнуть на хуй','пятикрылый шестихуй','рад дурак, что хуй велик',
                  'раззуделася пизда','разъёб','разъёба','разъебай','разъебайский','разъебалово','разъебать','разъебон',
                  'разъебончик','разъебти','разъебуха','разъёбывать','раскурочить ебало','распидорасить','распиздеть',
                  'распиздеться','распиздун','распиздяй','распиздяйка','распиздяйский','распиздяйство','распроебать',
                  'расхлестать ебосос','расхуюжить','расхуяривать','расхуяриваться','расхуярить','расхуяриться','расхуячить',
                  'руки под хуй заточены','с какого хуя','с нихуя','с пиздий волос','с хуёчка капает','с хуя ли','с хуя ль',
                  'свиноёб','свистни в хуй','сдуру можно и хуй сломать','семихуй','Скороёбишка','слово за слово, хуем по столу',
                  'слушать пиздой','смехуёчки','смешить пизду','смешно дураку, что хуй на боку','смотреть как пизда на лимон',
                  'собственными ёбалдами подавишься','сосать хуй','социоблядь','спиздануть','спиздеть','спиздивший','спиздить',
                  'спиздиться','спизженный','сравнить хуй с пальцем','старая пизда','старый хуй','сто хуёв — большая куча?',
                  'страхоёбина','страхопиздище','сунуть в еблецо','схуячить','сцуко','съебаторы','съебаторы врубить','съебать',
                  'съебаться','съебошить','съёбывать','съёбываться','таких друзей — за хуй да в музей','таракану хуй приделать',
                  'то ли лыжи не едут, то ли я ебанутый','толстопиздый','торговать пиздой','торговать пиздой и жопой','тупиздень',
                  'ты что, хуем подавился?','угощать пиздой','уёба','уебан','уёбан','уебанский','уебать','уебаться','уёбище',
                  'уёбищность','уёбищный','уёбок','уёбский','уёбство','уебти','уёбывать','уёбыш','упиздить','упиздовать',
                  'ухуякивать','ухуярить','хитровыебанность','хитровыебанный','хоть бы хуй','хоть самого в жопу сиськой еби',
                  'хрен моржовый','худоёбина','хуев','хуева туча','хуевастый','хуевато','хуеватый','хуёвее','хуёвей',
                  'хуёвенький','хуевертить','хуёвина','хуёвина с морковиной','хуёвинка','хуёвить','хуёвничать','хуёво',
                  'хуёвость','хуевый','хуёвый','хуеглот','хуеглотина','хуежоп','хуежопость','хуежопый','хуёк',
                  'хуем груши выколачивать','хуем груши околачивать','хуем груши сбивать','хуем задеть',
                  'хуем запугать','хуем наградить','хуем разворочена','хуем толкнуть','хуем угостить',
                  'хуем угощать','хуем умыть','хуё-моё','хуемразь','хуеносец','хуеньки','хуепас',
                  'хуеплёт','хуеплётка','хуепутало','хуерга','хуерговина','хуерыга','хуерык','хуерылый','хуесос',
                  'хуесос ёбаный','хуесосина','хуесосинг','хуесосить','хуесоска','хуета','хуета на хуете','хуетень',
                  'хуетища','хуеть','хуец','хуёчек','хуёчек плачет жгучею слезой','хуи','хуи гну','хуи жевать',
                  'хуи пинать','хуила','хуилище','хуило','хуило моржовый','хуильник','хуина','хуинушка','хуистый',
                  'хуишечко','хуишко','хуище','хуй','хуй без соли доедать','хуй в кожаном пальто','хуй в пальто',
                  'хуй важный','хуй вставить по яички','хуй гну','хуй да ни хуя','хуй до дома доволочь','хуй его знает',
                  'хуй забить','хуй знает','хуй колом','хуй ли','хуй моржовый','хуй на блюде','хуй на колёсиках','хуй на ны',
                  'хуй на рыло','хуй на рыло, чтоб сердце не ныло','хуй наны','хуй не стоит','хуй ночевал','хуй один',
                  'хуй по всей морде','хуй показать','хуй проссышь','хуй с горы','хуй с горы — большая скорость?',
                  'хуй с горы — какая скорость?','хуй с тобой','хуй сосали комбайнёры','хуй сосать','хуй там','хуй тебе',
                  'хуй тебе в рыло','хуй тебе на воротник','хуй тебе на рыло','хуй трёхколёсный','хуй ушастый','хуй царапать',
                  'Хуйкино','хуйлан','хуйло','хуйлыга','хуйнуть','хуйню спороть','хуйня','хуйня на постном масле','хуй-перехуй',
                  'хуйство','хуле','хули','хулиард','хуль','хуля','хуля-перехуля','хуюжить','хуючить','хуюшки','хуя с два','хуяк',
                  'хуяка','хуякать','хуякнуть','хуякнуться','хуякс','хуями обложить','хуяра','хуярить','хуяриться','хуястый',
                  'хуячить','хуяшить','через три пизды колено','через хуй зари не видит','через хуй кинуть','чешется пиздища',
                  'чешется пиздище','что за ёб твою мать?','шароёбить','шароёбиться','шароёбство',
                  'шахтёрам — кресты, инженерам — пизды, директору — нары, владельцу — Канары',
                  'это у вас в пизде квас, а у нас — в бочках','ябать','ябти','ябтись','японский','яти']

In [6]:
def preprocess_text(text):
    data["toxic"] = data["toxic"].apply(int)
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('russian')]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

# Добавление флага токсичности на основе списка токсичных слов
def is_toxic(comment):
    comment_lower = comment.lower()
    return int(any(re.search(pattern, comment_lower) for pattern in toxic_patterns))

In [ ]:
# Применение предобработки к каждому комментарию
data['cleaned_comment'] = data['comment'].apply(preprocess_text)

# Добавление признака токсичности на основе списка токсичных слов
data['toxic_flag'] = data['comment'].apply(is_toxic)

# Если любой из флагов токсичности установлен, маркируем комментарий как токсичный
data['toxic_combined'] = data.apply(lambda row: max(row['toxic'], row['toxic_flag']), axis=1)


In [ ]:
data['toxic_combined']

In [18]:
# Векторизация текста
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(data['cleaned_comment'])
# Признак токсичности
y = data['toxic_combined']

<14412x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 125317 stored elements in Compressed Sparse Row format>

In [9]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Обучение модели XGBoost
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [10]:
# Оценка точности модели
# Предсказание на тестовой выборке
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.7395074575095387
              precision    recall  f1-score   support

           0       0.70      0.96      0.81      1661
           1       0.88      0.45      0.59      1222

    accuracy                           0.74      2883
   macro avg       0.79      0.70      0.70      2883
weighted avg       0.78      0.74      0.72      2883



In [11]:
# Использование модели для предсказаний
def predict_toxicity(comment):
    cleaned_comment = preprocess_text(comment)
    comment_vector = vectorizer.transform([cleaned_comment])
    model_prediction = model.predict(comment_vector)
    toxic_flag = is_toxic(comment)
    combined_prediction = max(model_prediction[0], toxic_flag)
    return "Toxic" if combined_prediction == 1 else "Not Toxic"

In [23]:
# Пример использования модели
new_comment = " русским не объяснишь. как речь про СВО идёт,то они требуют других народов помощи,тк «это наша страна»,а в остальное время это их страна"
print(predict_toxicity(new_comment))

Not Toxic
